In [254]:
import pandas as pd
import numpy as np


---
### Загрузка данных

In [255]:
project_1_data = pd.read_csv('data/dst-3.0_16_1_hh_database.csv', sep=';')   # импорт из файла
hh_data = project_1_data.copy()                                              # создаём копию для работы
display(hh_data.head(2))
display(hh_data)
display(hh_data.info())

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,"Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано


,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,"Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано
2,"Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано
3,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано
4,"Женщина , 26 лет , родилась 3 марта 1993",140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,Опыт работы 5 лет 7 месяцев Региональный мене...,Мармелад,Менеджер по продажам,Высшее образование 2015 Кгу Психологии и педаг...,22.04.2019 10:32,Не указано
...,...,...,...,...,...,...,...,...,...,...,...,...
44739,"Мужчина , 30 лет , родился 17 января 1989",50000 руб.,"Финансист, аналитик, экономист, бухгалтер, мен...","Тверь , готов к переезду (Москва, Химки) , гот...",полная занятость,"полный день, удаленная работа","Опыт работы 7 лет 7 месяцев Финансист, аналит...","ООО ""IAS"" (независимый участник объединения Ru...",Руководитель субгруппы,Высшее образование 2015 Московский гуманитарн...,22.04.2019 12:32,Не указано
44740,"Мужчина , 27 лет , родился 5 марта 1992",39000 руб.,"Системный администратор, IT-специалист","Липецк , готов к переезду , готов к командировкам","проектная работа, частичная занятость, полная ...","удаленная работа, гибкий график, полный день, ...","Опыт работы 7 лет Системный администратор, IT...",ИП Пестрецов,Предприниматель,Высшее образование (Бакалавр) 2016 Воронежски...,22.04.2019 13:11,Не указано
44741,"Женщина , 48 лет , родилась 26 декабря 1970",40000 руб.,"Аналитик данных, Математик","Челябинск , готова к переезду , готова к редки...",полная занятость,"полный день, удаленная работа",Опыт работы 21 год 5 месяцев Январь 1998 — по...,"ОАО «ЧМК», Исследовательско-Технологический Це...",Начальник группы аналитики,Высшее образование 2000 Южно-Уральский госуда...,09.04.2019 05:07,Не указано
44742,"Мужчина , 24 года , родился 6 октября 1994",20000 руб.,Контент-менеджер,"Тамбов , не готов к переезду , не готов к кома...","частичная занятость, полная занятость",удаленная работа,Опыт работы 3 года 10 месяцев Контент-менедже...,IQ-Maxima,Менеджер проектов,Высшее образование 2015 Тамбовский государств...,26.04.2019 14:25,Имеется собственный автомобиль


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44744 entries, 0 to 44743
Data columns (total 12 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   Пол, возраст                     44744 non-null  object
 1   ЗП                               44744 non-null  object
 2   Ищет работу на должность:        44744 non-null  object
 3   Город, переезд, командировки     44744 non-null  object
 4   Занятость                        44744 non-null  object
 5   График                           44744 non-null  object
 6   Опыт работы                      44576 non-null  object
 7   Последнее/нынешнее место работы  44743 non-null  object
 8   Последняя/нынешняя должность     44742 non-null  object
 9   Образование и ВУЗ                44744 non-null  object
 10  Обновление резюме                44744 non-null  object
 11  Авто                             44744 non-null  object
dtypes: object(12)
memory usage: 4.1+

None

In [256]:
hh_data['Опыт работы'].nunique()

44413

In [257]:
hh_data['Ищет работу на должность:'].value_counts()

Системный администратор                                                  3099
Инженер                                                                   966
Аналитик                                                                  809
Менеджер проектов                                                         745
Руководитель проекта                                                      718
                                                                         ... 
Системный администратор Linux, DevOps, Разработчик Golang                   1
Инженер КИПиА,инженер АСУТП ,видеоинженер                                   1
SEO-специалист | Специалист по интернет-рекламе | Интернет-маркетолог       1
Специалист технических систем безопастности                                 1
Аналитик данных, Математик                                                  1
Name: Ищет работу на должность:, Length: 14929, dtype: int64

---

### Дополнительные функции

In [258]:
def f_razdelenie_stroki(stroka, razdelitel=','):
    '''Разбивает строку по заданному разделителю (по умолчанию - запятая),
    в полученных строках удаляются пробелы в начале и конце.
    Возвращается список строк.    
    '''
    spisok_stroka = stroka.lower()
    spisok_stroka = spisok_stroka.split(razdelitel)
    for n in range(0, len(spisok_stroka)):
        spisok_stroka[n] = spisok_stroka[n].strip()
    return spisok_stroka

---
### Анализ столбца пол/возраст/др

In [259]:
def f_pol(stroka, razdelitel=','):                                   # функция выделения данных пола
    '''Функция из данных столбца 'Пол, возраст' выбирает
    "мужчина" или "женщина" и возвращает М или Ж.
    Если не найдены - возвращает 0
    '''
    pol = f_razdelenie_stroki(stroka, razdelitel)[0]
    if pol == 'мужчина':
        pol = 'М'
    elif pol == 'женщина':
        pol = 'Ж'
    else:
        pol = 0
    return pol

def f_vozrast(stroka, razdelitel=','):                               # функция выделения данных возраста
    '''Функция из данных столбца 'Пол, возраст' выбирает
    возраст и возвращает в виде целоги числа.
    Если не найден - возвращает 0
    '''
    vozrast = f_razdelenie_stroki(stroka, razdelitel)[1]
    spisok_vozrast = vozrast.split()
    if len(spisok_vozrast) == 2:
        return int(spisok_vozrast[0])
    return 0

def f_dr(stroka, razdelitel=','):                                    # функция выделения данных ДР
    '''Функция из данных столбца 'Пол, возраст' выбирает
    дату рождения и возвращает в формате даты.
    Если не найдены - возвращает 0
    '''
    dr = f_razdelenie_stroki(stroka, razdelitel)[2]
    spisok_pol_dr = dr.split()
    month_list = ['января', 'февраля', 'марта', 'апреля', 'мая', 'июня',
           'июля', 'августа', 'сентября', 'октября', 'ноября', 'декабря']
    if len(spisok_pol_dr) == 4:
        spisok_dr = spisok_pol_dr[1:]
        month_str = spisok_dr[1]
        if month_str in month_list:                                  # меняем месяц прописью на числовое значение
            month_ch = str(month_list.index(month_str)+1)
            spisok_dr[1] = month_ch
            dr_dt = pd.to_datetime('.'.join(spisok_dr), dayfirst=True)
        return dr_dt
    return 0

def f_pol_dop(stroka, razdelitel=','):                               # функция выделения пола из ДР
    '''Функция из данных о рождении столбца 'Пол, возраст'
    получает пол и возвращает М или Ж.
    Если не найдены - возвращает 0
    '''
    dr = f_razdelenie_stroki(stroka, razdelitel)[2]
    spisok_pol_dr = dr.split()
    if len(spisok_pol_dr) == 4:
        spisok_pol_dop = spisok_pol_dr[0]
        if spisok_pol_dop == 'родился':
            pol_dop = 'М'
        elif spisok_pol_dop == 'родилась':
            pol_dop = 'Ж'
        else:
            pol_dop = 0
        return pol_dop
    return 0


hh_data_dr = hh_data[['Пол, возраст', 'Обновление резюме']].copy()   # создаём временную копию данных для обработки

hh_data_dr['Пол'] = hh_data_dr['Пол, возраст'].apply(f_pol)          # Сравним указанный пол с полученным по анализу данных о рождении
#hh_data_dr['Пол (доп.)'] = hh_data_dr['Пол, возраст'].apply(f_pol_dop)
#hh_data_dr['Дельта пол'] = hh_data_dr['Пол'] + hh_data_dr['Пол (доп.)']
display('Отклонения указанного пола с полученным из данных о дате рождения',
        (hh_data_dr['Пол'] + hh_data_dr['Пол, возраст'].apply(f_pol_dop)).value_counts())

hh_data_dr['ДР'] = hh_data_dr['Пол, возраст'].apply(f_dr)            # Сравним указанный возраст с вычисляемым по дате рождения
hh_data_dr['Возраст'] = hh_data_dr['Пол, возраст'].apply(f_vozrast)
#hh_data_dr['Возраст (доп.)'] = (pd.to_datetime(hh_data_dr['Обновление резюме'], dayfirst=True) - hh_data_dr['ДР']).dt.days//364.25
#hh_data_dr['Дельта возраста'] = hh_data_dr['Возраст'] - hh_data_dr['Возраст (доп.)']
display('Отклонения указанного возраста с вычисляемым по дате рождения',
        (hh_data_dr['Возраст'] - (pd.to_datetime(hh_data_dr['Обновление резюме'], dayfirst=True) - hh_data_dr['ДР']).
         dt.days//364.25).value_counts())

#display(hh_data_dr)
#display(hh_data_dr['Пол'].value_counts())
#display(hh_data_dr['Пол (доп.)'].value_counts())
#display(hh_data_dr['Возраст'].value_counts())
#display(hh_data_dr['ДР'].value_counts())


'Отклонения указанного пола с полученным из данных о дате рождения'

ММ    36211
ЖЖ     8533
dtype: int64

'Отклонения указанного возраста с вычисляемым по дате рождения'

 0.0    41197
-1.0     2545
 1.0     1002
dtype: int64

* Данные указания пола полностью подтверждается анализом данных о дате рождения.

* Указанный возраст отличается от вычисляемого по дате рождения не более чем на 1 год.  
Это вполне может быть связано с ошибками заполнения возраста количеством полных/неполных лет,  
смещением даты обновления резюме от даты начального заполнения и не является критическим отклонением.  
Будем считать данные возраста заполненными правильными.

* Теперь переместим обработанные данные в основную таблицу и избавимся от лишних признаков.

In [260]:
hh_data_dr.drop(['Пол, возраст','Обновление резюме','ДР'],       # удаляем лишние столбцы из временной таблицы
                axis=1, inplace=True)
hh_data_dr['Пол'] = hh_data_dr['Пол'].astype('category')         # столбцу "Пол" зададим тип "категории"
display(hh_data_dr.head())
display(hh_data_dr.info())

,Пол,Возраст
0,М,39
1,М,60
2,Ж,36
3,М,38
4,Ж,26


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44744 entries, 0 to 44743
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   Пол      44744 non-null  category
 1   Возраст  44744 non-null  int64   
dtypes: category(1), int64(1)
memory usage: 393.5 KB


None

In [261]:
hh_data = hh_data.join(hh_data_dr)                       # объединяем таблицы
hh_data.drop(['Пол, возраст'], axis=1, inplace=True)     # удаляем столбец
display(hh_data.head())
display(hh_data.info())

,ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто,Пол,Возраст
0,29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль,М,39
1,40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано,М,60
2,20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано,Ж,36
3,100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано,М,38
4,140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,Опыт работы 5 лет 7 месяцев Региональный мене...,Мармелад,Менеджер по продажам,Высшее образование 2015 Кгу Психологии и педаг...,22.04.2019 10:32,Не указано,Ж,26


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44744 entries, 0 to 44743
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   ЗП                               44744 non-null  object  
 1   Ищет работу на должность:        44744 non-null  object  
 2   Город, переезд, командировки     44744 non-null  object  
 3   Занятость                        44744 non-null  object  
 4   График                           44744 non-null  object  
 5   Опыт работы                      44576 non-null  object  
 6   Последнее/нынешнее место работы  44743 non-null  object  
 7   Последняя/нынешняя должность     44742 non-null  object  
 8   Образование и ВУЗ                44744 non-null  object  
 9   Обновление резюме                44744 non-null  object  
 10  Авто                             44744 non-null  object  
 11  Пол                              44744 non-null  category
 12  Возр

None

---
### Анализ столбца образования

In [262]:
hh_data_obr = hh_data[['Образование и ВУЗ']].copy()   # создаём временную копию данных образования для обработки

def f_str_obr(stroka):      # функция выделения типа образования
    ''' Функция находит в строке слово "образование", и возвращает
    начальную часть строки перед найденным словом.
    Если слово "образование" не найдено - возвращается 0    
    '''
    spisok_obr = stroka.split()
    if 'образование' in spisok_obr:
        spisok_obr = spisok_obr[:spisok_obr.index('образование')]
        stroka_itog = ' '.join(spisok_obr)
        return stroka_itog.lower()
    return 0

#s_proba = hh_data_obr['Образование и ВУЗ'][0]
#display(s_proba.split(' '))
#display(s_proba.split())

hh_data_obr['Образование'] = hh_data_obr['Образование и ВУЗ'].apply(f_str_obr)  # создаём столбец с типом образования
display(hh_data_obr['Образование'].value_counts())                              # проверяем итоговый результат
hh_data_obr.drop(['Образование и ВУЗ'], axis=1, inplace=True)                   # удаляем столбец с начальными данными

hh_data = hh_data.join(hh_data_obr)                                             # объединяем таблицы
display(hh_data.info())
hh_data.drop(['Образование и ВУЗ'], axis=1, inplace=True)                       # удаляем столбец 'Образование и ВУЗ'
hh_data['Образование'] = hh_data['Образование'].astype('category')              # столбцу "Образование" зададим тип "категории"
display(hh_data.head(3))
display(hh_data.info())


высшее                 33863
среднее специальное     5765
неоконченное высшее     4557
среднее                  559
Name: Образование, dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44744 entries, 0 to 44743
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   ЗП                               44744 non-null  object  
 1   Ищет работу на должность:        44744 non-null  object  
 2   Город, переезд, командировки     44744 non-null  object  
 3   Занятость                        44744 non-null  object  
 4   График                           44744 non-null  object  
 5   Опыт работы                      44576 non-null  object  
 6   Последнее/нынешнее место работы  44743 non-null  object  
 7   Последняя/нынешняя должность     44742 non-null  object  
 8   Образование и ВУЗ                44744 non-null  object  
 9   Обновление резюме                44744 non-null  object  
 10  Авто                             44744 non-null  object  
 11  Пол                              44744 non-null  category
 12  Возр

None

,ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Обновление резюме,Авто,Пол,Возраст,Образование
0,29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,16.04.2019 15:59,Имеется собственный автомобиль,М,39,неоконченное высшее
1,40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",12.04.2019 08:42,Не указано,М,60,высшее
2,20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,16.04.2019 08:35,Не указано,Ж,36,среднее специальное


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44744 entries, 0 to 44743
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   ЗП                               44744 non-null  object  
 1   Ищет работу на должность:        44744 non-null  object  
 2   Город, переезд, командировки     44744 non-null  object  
 3   Занятость                        44744 non-null  object  
 4   График                           44744 non-null  object  
 5   Опыт работы                      44576 non-null  object  
 6   Последнее/нынешнее место работы  44743 non-null  object  
 7   Последняя/нынешняя должность     44742 non-null  object  
 8   Обновление резюме                44744 non-null  object  
 9   Авто                             44744 non-null  object  
 10  Пол                              44744 non-null  category
 11  Возраст                          44744 non-null  int64   
 12  Обра

None

In [263]:
# Сколько процентов женских резюме представлено в наших данных? Ответ округлите до сотых.
round(hh_data[hh_data['Пол'] == 'Ж']['Пол'].count()/hh_data['Пол'].count() * 100, 2)

19.07

In [264]:
#Чему равен средний возраст соискателей? Ответ округлите до десятых.
round(hh_data['Возраст'].mean(), 1)

32.2

---
### Анализ признака "Опыт работы"

In [265]:
def f_op_rabot(stroka):
    '''Функция из данных опыта работы вычисляет и возвращает опыт
    работы в месяцах, иниче возвращает NaN
    '''
    if stroka is np.nan:                               # Возврат NaN, если данные отсутствуют
        return np.nan
    spisok_op_rabot = stroka.split()[:6]
    n = len(spisok_op_rabot)-1
    if n < 3:                                          # Возврат NaN, если данные отсутствуют
        return np.nan
    mes_list = ['месяц','месяца','месяцев']
    god_list = ['год','года','лет']
    mes_ch = 0
    god_ch = 0
    while n > 0:
        if spisok_op_rabot[n] in mes_list:
            mes_ch = int(spisok_op_rabot[n-1])
            n-=2
        if spisok_op_rabot[n] in god_list:
            god_ch = int(spisok_op_rabot[n-1])
            break
        n-=1
    mes_ch = mes_ch + god_ch*12
    return mes_ch


hh_op_rabot = hh_data[['Опыт работы']].copy()           # создаём временную копию данных опыта для обработки
hh_op_rabot['Опыт работ (месяц)'] = hh_op_rabot['Опыт работы'].apply(f_op_rabot)
#display(hh_op_rabot)
display(hh_op_rabot.info())

display(hh_op_rabot['Опыт работ (месяц)'].median())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44744 entries, 0 to 44743
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Опыт работы         44576 non-null  object 
 1   Опыт работ (месяц)  44574 non-null  float64
dtypes: float64(1), object(1)
memory usage: 699.2+ KB


None

100.0

In [266]:
hh_op_rabot.drop(['Опыт работы'],                         # удаляем лишние столбцы из временной таблицы
                axis=1, inplace=True)
hh_data = hh_data.join(hh_op_rabot)                       # объединяем таблицы
hh_data.drop(['Опыт работы'], axis=1, inplace=True)       # удаляем столбец
display(hh_data.head())
display(hh_data.info())

,ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Последнее/нынешнее место работы,Последняя/нынешняя должность,Обновление резюме,Авто,Пол,Возраст,Образование,Опыт работ (месяц)
0,29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...","МАОУ ""СОШ № 1 г.Немана""",Системный администратор,16.04.2019 15:59,Имеется собственный автомобиль,М,39,неоконченное высшее,202.0
1,40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",12.04.2019 08:42,Не указано,М,60,высшее,233.0
2,20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,ПАО Сбербанк,Кассир-операционист,16.04.2019 08:35,Не указано,Ж,36,среднее специальное,123.0
3,100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",OpenSoft,Инженер-программист,08.04.2019 14:23,Не указано,М,38,высшее,225.0
4,140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,Мармелад,Менеджер по продажам,22.04.2019 10:32,Не указано,Ж,26,высшее,67.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44744 entries, 0 to 44743
Data columns (total 13 columns):
 #   Column                           Non-Null Count  Dtype   
---  ------                           --------------  -----   
 0   ЗП                               44744 non-null  object  
 1   Ищет работу на должность:        44744 non-null  object  
 2   Город, переезд, командировки     44744 non-null  object  
 3   Занятость                        44744 non-null  object  
 4   График                           44744 non-null  object  
 5   Последнее/нынешнее место работы  44743 non-null  object  
 6   Последняя/нынешняя должность     44742 non-null  object  
 7   Обновление резюме                44744 non-null  object  
 8   Авто                             44744 non-null  object  
 9   Пол                              44744 non-null  category
 10  Возраст                          44744 non-null  int64   
 11  Образование                      44744 non-null  category
 12  Опыт

None